# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database **data.sqlite**.

In [ ]:
import pandas as pd
import sqlite3
conn = sqlite3.Connection('data.sqlite')
pd.read_sql("SELECT name FROM sqlite_master WHERE type ='table'",conn)

## Display the names of all the employees in Boston.
Hint: join the employees and offices tables.

In [ ]:
pd.read_sql("""
SELECT lastName,firstName
FROM employees
JOIN offices USING(officeCode)
WHERE city='Boston'
""",conn)

## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [ ]:
pd.read_sql("""
SELECT city,count(employeeNumber) as employeeCount
FROM offices
LEFT JOIN employees USING (officeCode)
GROUP BY city
""",conn)

## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold
pd.read_sql("""
SELECT productName, textDescription, firstName, lastName, sum(QuantityOrdered) numSold
FROM productLines
JOIN products USING(productLine)
JOIN orderdetails USING(productCode)
JOIN orders USING(orderNumber)
JOIN customers USING(customerNumber)
JOIN employees ON salesRepEmployeeNumber=employeeNumber
GROUP BY productName,firstName,lastName
""",conn)

In [ ]:
# Best-performing offices
pd.read_sql("""
SELECT o.city office, sum(QuantityOrdered) numSold, printf('$%.2f',sum(QuantityOrdered * priceEach)) sales
FROM productLines
JOIN products USING(productLine)
JOIN orderdetails USING(productCode)
JOIN orders USING(orderNumber)
JOIN customers USING(customerNumber)
JOIN employees ON salesRepEmployeeNumber=employeeNumber
JOIN offices o USING(officeCode)
GROUP BY o.city
ORDER BY sum(QuantityOrdered * priceEach) DESC
""",conn)

In [ ]:
# Top customers listed by sales
pd.read_sql("""
SELECT customerName, sum(QuantityOrdered) numSold, printf('$%.2f',sum(QuantityOrdered * priceEach)) sales
FROM productLines
JOIN products USING(productLine)
JOIN orderdetails USING(productCode)
JOIN orders USING(orderNumber)
JOIN customers USING(customerNumber)
GROUP BY customerNumber
ORDER BY sum(QuantityOrdered * priceEach) DESC
LIMIT 20
""",conn)

In [ ]:
# Order volume by year by office
pd.read_sql("""
SELECT strftime('%Y',shippedDate) year, o.city office,count(orderNumber) orders
FROM orders 
JOIN customers USING(customerNumber)
JOIN employees ON salesRepEmployeeNumber=employeeNumber
JOIN offices o USING(officeCode)
WHERE year IS NOT NULL
GROUP BY year,office
ORDER BY year,orders DESC
""",conn)

In [ ]:
# sqlite doesn't have a pivot table so
# Order volume by year by office
pd.read_sql("""
SELECT o.city office,
       count(case when strftime('%Y',shippedDate) = '2003' then orderNumber end) AS '2003',
       count(case when strftime('%Y',shippedDate) = '2004' then orderNumber end) AS '2004',
       count(case when strftime('%Y',shippedDate) = '2005' then orderNumber end) AS '2005'
FROM orders 
JOIN customers USING(customerNumber)
JOIN employees ON salesRepEmployeeNumber=employeeNumber
JOIN offices o USING(officeCode)
GROUP BY office
""",conn).set_index("office")

# Of course could 'easily' do this in pandas instead

In [ ]:
conn.close()

## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!